In [152]:
%run gower_function-v6.4.ipynb
from pymongo import MongoClient
from random import randint
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from scipy.spatial.distance import pdist,squareform, jaccard, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.s

In [153]:
client = MongoClient(port=27017)
db = client.weare    

In [154]:
student_queries = db.students.find({})
students = pd.DataFrame(list(student_queries))

In [155]:
students['id']=students.Email.str.split("@", n=1, expand=True)[0]
if '_id' in students.columns:
    del students['_id']

In [156]:
#drop rows with 5 or more null values
students.dropna(thresh=5, inplace=True)
students = students.replace('', np.nan)

In [157]:
#number of courses, credits are too dirty as many entered text instead of a number
drop_columns = ["surveyCompletion", "Duration (in seconds)", "NumCourses", "transferCredits", "NumTranCredits",
                "WhyProfile", "otherChannels", "otherEmploy", "NonUS", "otherIndusry", "KnownThroughProfile",
                "otherEth", "PPLinPerson", "otherEmail"]
#only drop these columns if they actually exist in the data
drop_columns = [col for col in drop_columns if col in students.columns]
students.drop(columns=drop_columns, inplace=True)
#names of all the soc columns
SOCs = ['SOC'+str(x+1) for x in range(10)]
#names of all the CCE columns
CCEs = ['CCE'+str(x+1) for x in range(24)]
#mapping from each text column to ordinal values
#format is array of dicts containing keys 'col' and 'mapping'
ordinal_cols_mapping = []

In [158]:
#the following are mappings from textual column values to numberical ones
important_scale = [
        ('Extremely important', 7),
        ('Pretty important', 6),
        ('Fairly important', 5),
        ('Moderately important', 4),
        ('Somewhat important', 3),
        ('A little important', 2),
        ('Not at all important', 1)
]

interest_scale = [
    ('Extremely interested', 5),
    ('Rather interested', 4),
    ('Somewhat interested', 3),
    ('A bit of interest', 2),
    ('Not at all interested', 1)
]

Agree_scale = [
        ('Strongly agree', 5),
        ('Somewhat agree', 4),
        ('Neither agree nor disagree', 3),
        ('Somewhat disagree', 2),
        ('Strongly disagree', 1)
]

In [159]:
#fill in the mapping array 
for SOC in SOCs:
    ordinal_cols_mapping.append({
        "col": SOC,
        "mapping": Agree_scale
    })

for CCE in CCEs:
    ordinal_cols_mapping.append({
        "col": CCE,
        "mapping": Agree_scale
    })

for PeerInfo in ["PeerAvail", "PeerProfession", "PeerEdu", "PeerDetails"]:
    ordinal_cols_mapping.append({
        "col": PeerInfo,
        "mapping": important_scale
    })

ordinal_cols_mapping.append({
    "col": "Mconnected",
    "mapping": interest_scale
})

In [160]:
encoder = ce.OrdinalEncoder(mapping=ordinal_cols_mapping, return_df=True)
df = encoder.fit_transform(students)
#Create new columns by taking the mean of related columns for each row
df['SOC_f'] = df.apply(lambda row: np.mean([row['SOC9'], row['SOC10']]), axis=1)
df['SOC_id'] = df.apply(lambda row: np.mean([row['SOC1'], row['SOC2'], row['SOC3'], row['SOC4'], row['SOC5']]), axis=1)
df['CCE_IR'] = df.apply(lambda row: np.mean([row['CCE13'], row['CCE10'], row['CCE19'], row['CCE9']]), axis=1)
df['CCE_Coor'] = df.apply(lambda row: np.mean([row['CCE24'], row['CCE20'], row['CCE22'], row['CCE21'], row['CCE17']]), axis=1)
df['CCE_SS'] = df.apply(lambda row: np.mean([row['CCE1'], row['CCE2'], row['CCE3'], row['CCE4']]), axis=1)
df.drop(columns=CCEs + SOCs, inplace=True)

In [161]:
none_i = re.compile(r'none', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
none_i = re.compile(r'zero', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
stringany = re.compile(r'[a-zA-Z ()]+', flags=re.IGNORECASE)
print(f'Before digit processing, kids are {df.kids.unique()}')
df.kids = df['kids'].replace(stringany, '')

Before digit processing, kids are [ 0.  1.  2.  3.  4. nan  5.  9.]


In [162]:
categorical_cols = ["gender", "InUS", "ethnicity", "Usstate", "marrital", "employment", "industry"]
categorical_cols = [c for c in categorical_cols if c in df.columns]
df_c_mode = df[categorical_cols].mode()
print(f'mode listed are \n{df_c_mode.iloc[0]}')
print(len(df_c_mode))
for col in categorical_cols+['kids']:
    df[co].fillna(df[col].mode().iloc[0], inplace=True)


mode listed are 
gender                         Female
InUS                              Yes
ethnicity                   Caucasian
Usstate                  Pennsylvania
marrital                      Married
employment    Working (paid employee)
industry         Educational services
Name: 0, dtype: object
1


In [163]:
# fill with mode, mean, or median
df_mode, df_mean, df_median = df.mode().iloc[0], df.mean(), df.median()
df.fillna(df_median, inplace=True)

In [170]:
numeric_columns = df._get_numeric_data().columns
col_is_categorical = [col not in numeric_columns for col in df.columns]
distances = gower_distances(df, categorical_features=col_is_categorical)

/home/matt/.conda/envs/weare/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


In [204]:
def get_matches(index, distances, df, num_matches=1):
    best_scores = sorted(distances[index])[1: 1+num_matches]
    best_entries = [np.where(distances[index]==score)[0][0] for score in best_scores]
    og_user = df.iloc[[index]]
    best_users = [df.iloc[[entry]] for entry in best_entries]
    all_users = [og_user, *best_users]
    concat = pd.concat(all_users)
    return concat
def display_matches(distances, df, num_matches=10):
    frame = get_match(0, distances, df)
    for i in range(1, num_matches):
        match = get_matches(i, distances, df)
        frame = pd.concat([frame, match])
    return frame

In [205]:
display_matches(distances, df)
get_matches(0, distances, df, num_matches=10)

,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
0,NaN,No,Master degree,sxa459@psu.edu,Sana,72.94.219.150,Yes,Abdulateef,40.351395,-75.930603,...,Educational services,0.0,Married,14.0,sxa459,2.5,5.0,4.50,3.8,3.50
172,NaN,No,Master degree,deb77@psu.edu,Debra,73.187.239.247,Yes,Barshinger,39.893906,-76.557999,...,Educational services,0.0,Married,8.0,deb77,3.0,4.4,4.25,4.4,4.25
32,NaN,No,Bachelor degree,ALH19@PSU.EDU,Apryl,128.118.208.194,Yes,Hawn,40.784897,-77.847000,...,Educational services,0.0,Married,12.0,ALH19,2.0,4.0,4.00,4.0,4.00
40,NaN,No,Master degree,sab89@psu.edu,Sarah,74.193.239.88,Yes,Brown,31.134506,-93.180298,...,Educational services,2.0,Married,50.0,sab89,2.0,4.4,5.00,4.8,4.75
134,NaN,No,Master degree,cdh32@psu.edu,Carolyn,96.59.78.230,Yes,Holme,27.886703,-82.511703,...,Finance or insurance,0.0,Married,2.0,cdh32,2.0,4.2,4.50,3.4,4.50
89,NaN,No,Master degree,mlb593@psu.edu,Maria,165.189.65.30,Yes,Butters,43.020798,-89.360001,...,Educational services,0.0,Married,10.0,mlb593,1.0,3.6,4.50,4.4,3.75
93,NaN,No,Master degree,jlc672@psu.edu,Jennifer,72.69.210.247,Yes,Cogen,40.714294,-74.005997,...,"Arts, entertainment or recreation",0.0,Single/Never Married,41.0,jlc672,2.5,3.2,4.50,3.8,3.50
111,NaN,No,Master degree,skg143@psu.edu,Susan,216.15.78.209,Yes,Gair,40.657104,-75.502998,...,Educational services,0.0,Married,91.0,skg143,2.0,4.6,4.50,3.8,3.75
331,No,No,Bachelor degree,cxd415@psu.edu,Caitlin,67.183.131.121,Yes,Davis,47.025696,-122.796700,...,Educational services,0.0,Married,29.0,cxd415,2.5,4.2,4.75,4.4,4.50
346,NaN,No,Bachelor degree,dmd6002@psu.edu,Denise,107.77.202.214,Yes,Dollarton,39.735107,-75.668404,...,Educational services,4.0,Single/Never Married,6.0,dmd6002,1.0,3.8,4.50,4.6,3.75
